# Programming Exercise: Robot Localization

#### Cancer Test Example
Let us apply Bayes' Rule to a Cancer Test Example which is a very common example in Statistic classes.
Suppose, there is a certain type of cancer which is extremely rare.

Let,
\begin{equation}
P(C) = 0.001
\end{equation}
In other words, $1$ in every $1000$ people has this cancer. Can you compute the probability of not having cancer?
\begin{equation}
P(\neg C) = ?
\end{equation}

The test for cancer is not $100\%$ accurate. Let us say, probability of the test being positive given someone has cancer is,
\begin{equation}
P(pos | C) = 0.8
\end{equation}
Can you compute the probability that the test is negative given someone has cancer?
\begin{equation}
P(neg | C) = ?
\end{equation}

Similarly, the probability of a test being positive given that someone has no cancer is $P(pos | \neg C) = 0.1$. Can you compute the probability that the test is negative given someone has no cancer?
\begin{equation}
P(neg | \neg C) = ?
\end{equation}

Can you compute the following?
\begin{equation}
P(C | pos) = ?
\end{equation}
Interpret the quantity you computed in words!

What we computed is very interesting. We computed probability of having cancer given the test results and how likely the disease is.

# Robot Localization
In this exercise we will apply basic concepts of probability and statistics to help a robot localise itself. Robot localization is an important task for any mobile robot as it needs to know where in the world it is currently situated in.

You may think robot localization is a very easy problem as you can use GPS. But no! GPS has errors in localization upto a few metres which is very inaccurate for self driving cars.

Hence, the robot needs to use sensors to locate itself. And sensors are noisy! Even the motion of a robot is uncertain.

For the purposes of this exercise, let us assume that the world is a one dimensional world with $N$ grid cells. Each grid cell is colored either red or green which can be measured using a sensor located in the robot. However, the sensor measurements are noisy and hence we need to take a probabilitic approach to determine in which of these grid cells the robot is situated in at a given instant of time.

![image](images/1.png)

The robot can move either left or right in this **cyclic** world. This motion is uncertain as well prompting us again to take a probabilistic approach. For example you may instruct the robot to move $5$ cells. But it may move $4/6$ cells due to a wheel slip.

When the robot is switched on, it is in a state of maximum confusion. It doesn't know where it is situated. We will model this state of maximum confusion as uniform distribution. 
In other words, the robot has equal belief of being present in any of the cells.

Complete the below function to return an array where the $i^{th}$ array element is the probability that the robot is in the $i^{th}$ grid cell at time instant $0$ when the robot is switched on.

In [0]:
import numpy as np

In [0]:
# Modify the empty list, p, so that it becomes a UNIFORM probability
# distribution over n grid cells, as expressed in a list of 
# n probabilities.
def init_distribution(n):
    p = []
    # Add your code here
    p = np.full((n),1/n)
    # End your code here
    return p

In [5]:
init_distribution(5)

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [13]:
#Modify the code below so that the function sense, which 
#takes p and Z as inputs, will output the normalized 
#probability distribution, q, after multiplying the entries 
# in p according to the color in the corresponding cell in world,
# followed by normalization of the probability values
world=['green', 'red', 'red', 'green', 'green']
p = [0.2, 0.2, 0.2, 0.2, 0.2]
prob = 0.75
def sense(p, Z):
    q = []
    # Add your code here
    q= [prob * p[i] if (world[i] == Z)  else (1 - prob) * p[i] for i in range(len(world))]
    q = q/np.sum(q)
    # End your code here
    return q
            
sense(p, 'red')

array([0.11111111, 0.33333333, 0.33333333, 0.11111111, 0.11111111])

In [14]:
sense(p,'green')

array([0.27272727, 0.09090909, 0.09090909, 0.27272727, 0.27272727])

In [17]:
#Modify the code so that it updates the probability twice
#and gives the posterior distribution after both 
#measurements are incorporated. Make sure that your code 
#allows for any sequence of measurement of any length.

# DO NOT MODIFY THE SENSE FUNCTION. JUST CALL THE SENSE FUNCTION APPROPRIATELY

p=[0.2, 0.2, 0.2, 0.2, 0.2]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']

# Add your code here
for measurement in measurements:
   p = sense(p , measurement)
# End your code here
print(p)

[0.2 0.2 0.2 0.2 0.2]


In [35]:
#Program a function that returns a new distribution 
#q, shifted to the right by U units. If U=0, q should 
#be the same as p.

def move(p, U):
    q = []
    # ADD CODE HERE
    n = len(p)
    shift = U%n
    q = [*p,*p]
    q = np.array(q[n-shift:2*n-shift])
    # END CODE HERE
    return q

move([0.11, 0.33, 0.33, 0.11, 0.11], 1)

array([0.11, 0.11, 0.33, 0.33, 0.11])

**Solution:**

We can apply the Theorem of total probability at each grid cell given the robot instruction. For example, we are interested in computing the probability that the robot is at grid cell $2$. Now the robot can come here from the following gird cells:
- Grid cell $1$: The instruction is to move right by $2$, and hence a robot at cell $1$ can reach this grid cell with a probability of $0.1$(this is by undershooting)
- Grid cell $0$ : The instruction is to move right by $2$, and hence a robot at cell $0$ can reach this grid cell with a probability of $0.8$(this is case when robot motion is exact)
- Grid cell $4$ : The instruction is to move right by $2$, and hence a robot at cell $4$ can reach this grid cell with a probability of $0.1$(this is by overshooting). Remember that the world is cyclic.

Required Probability($P$) is given as follows:
\begin{equation}
P = 1.0 * 0.1 + 0.0 * 0.8 + 0.0 * 0.1 = 0.1
\end{equation}

In [36]:
#Modify the move function to accommodate the added 
#probabilities of overshooting or undershooting 
#the intended destination.

def move_prob(p, U):
    q = []
    # ADD CODE HERE
    if U == 0:
      return p
    under = 0.1
    over = 0.1
    precise = 0.8
    p = np.array(p)
    q=move(under*p,U-1)+move(precise*p,U)+move(over*p,U+1)
    # END CODE HERE
    return q

move_prob([0, 1.0, 0, 0, 0], 1)

array([0. , 0.1, 0.8, 0.1, 0. ])

In [37]:
# Write code that makes the robot move twice and then prints 
# out the resulting distribution, starting with the initial 
# distribution p = [0, 1, 0, 0, 0]
p = np.array([0, 1, 0, 0, 0])
# Call the function appropriately from here

# ADD CODE HERE
p=move_prob(p,1)
p=move_prob(p,1)
# END CODE HERE
p

array([0.01, 0.01, 0.16, 0.66, 0.16])

In [39]:
#write code that moves 1000 times and then prints the 
#resulting probability distribution.

p = [0, 1, 0, 0, 0]
# ADD CODE HERE
for i in range(1000):
  p = move_prob(p,1)
# END CODE HERE
p

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [41]:
#Given the list motions=[1,1] which means the robot 
#moves right and then right again, compute the posterior 
#distribution if the robot first senses red, then moves 
#right one, then senses green, then moves right again, 
#starting with a uniform prior distribution.

world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
motions = [1,1]

p = init_distribution(len(world))
# ADD CODE HERE
n = len(p)
for measurement , motion in zip(measurements,motions):
  p = sense(p,measurement)
  p = move_prob(p,1 if motion == 1 else n -1)
p

array([0.21157895, 0.15157895, 0.08105263, 0.16842105, 0.38736842])

[0.21157894736842106, 0.15157894736842106, 0.08105263157894738, 0.16842105263157894, 0.38736842105263164]